In [215]:
import numpy as np
import pandas as pd
import nltk
from nltk.stem import PorterStemmer

train_data_path = 'train.tsv'
test_data_path = 'valid.tsv'
output_path = 'output.txt'
stop_path = 'stopwords.txt'

In [216]:
train_df= pd.read_csv(train_data_path,header=None,sep='\t',quoting=3)
train_x= train_df[2]
train_y= train_df[1]
stop= pd.read_csv(stop_path,header=None)[0]
stop=stop.astype(str).tolist()

In [217]:
stemmer = PorterStemmer()

def preprocess_text(text):
    words = text.lower().split()
    filtered_words = [stemmer.stem(word) for word in words if word not in stop]
    return filtered_words

train_x= train_x.apply(preprocess_text)

In [218]:
label_to_index = {'pants-fire': 0, 'false': 1, 'barely-true': 2, 'half-true': 3, 'mostly-true': 4, 'true': 5}
index_to_label = {v: k for k, v in label_to_index.items()}


In [219]:
def create_feature_matrix(processed_texts, labels):
    vocabulary = set(word for text in processed_texts for word in text)
    vocab_size = len(vocabulary)
    
    word_to_index = {word: idx for idx, word in enumerate(vocabulary)}
    
    feature_matrix = np.zeros((len(processed_texts), vocab_size), dtype=int)
    class_count = np.zeros(len(set(labels)))

    for i, text in enumerate(processed_texts):
        for word in text:
            if word in word_to_index: 
                feature_matrix[i, word_to_index[word]] = 1
    
    encoded_labels = np.array([label_to_index[label] for label in labels])
    
    for label in labels:
        class_count[label_to_index[label]] += 1

    return feature_matrix, encoded_labels, class_count, word_to_index

X, Y, class_count, word_to_index= create_feature_matrix(train_x, train_y)

In [220]:
class BernoulliNaiveBayes:
    def __init__(self):
        self.class_priors = None
        self.feature_probs = None
        self.unseen_word_value_per_class = None

    def fit(self, X, y):
        self.class_priors = np.log(np.bincount(y) / len(y))
        self.feature_probs = np.zeros((len(self.class_priors), X.shape[1]))
        
        for c in range(len(self.class_priors)):
            X_c = X[y == c]
            self.feature_probs[c] = (X_c.sum(axis=0) + 1) / (X_c.shape[0] + 2)
        
        class_count = np.bincount(y)
        self.unseen_word_value_per_class = [1 / (2 + count) for count in class_count]

    def predict(self, X):
        eps = 1e-10
        log_feature_probs = np.log(self.feature_probs.T.clip(eps, 1 - eps))
        log_complement_probs = np.log((1 - self.feature_probs.T).clip(eps, 1 - eps))
        log_probs = X @ log_feature_probs + (1 - X) @ log_complement_probs + self.class_priors
        print(log_probs)
        return np.argmax(log_probs, axis=1)

    def predict_test(self, X_test, unseen_word_count):
        eps = 1e-10
        log_feature_probs = np.log(self.feature_probs.T.clip(eps, 1 - eps))
        log_complement_probs = np.log((1 - self.feature_probs.T).clip(eps, 1 - eps))
        log_probs = X_test @ log_feature_probs + (1 - X_test) @ log_complement_probs
        log_probs += self.class_priors
        
        for i in range(X_test.shape[0]): 
            for c in range(len(self.class_priors)): 
                if unseen_word_count[i] > 0:
                    unseen_contribution = np.log(self.unseen_word_value_per_class[c])
                    log_probs[i, c] += unseen_contribution 

        print(log_probs)
        return np.argmax(log_probs, axis=1)

model = BernoulliNaiveBayes()
model.fit(X, Y)

predictions = model.predict(X)

accuracy = np.mean(predictions == Y)
print(f'Accuracy: {accuracy * 100:.2f}%')

[[ -78.64936418  -65.55233884  -70.00651476  -68.98724047  -71.93176579
   -71.72923026]
 [-108.83755008  -96.12534969  -96.07170487  -93.57188172  -97.50562238
   -98.20088982]
 [ -99.57620777  -87.38163751  -86.82345498  -87.95758968  -85.75742993
   -88.86631469]
 ...
 [-112.38087176 -104.45694306 -103.95835085  -99.03386335 -105.10612606
  -104.51048885]
 [ -72.05993847  -60.251569    -64.82133122  -64.25033752  -61.33746182
   -64.08276089]
 [-117.27653833 -117.31261369 -120.12688176 -119.74731016 -120.92763637
  -119.11899867]]
Accuracy: 66.27%


In [221]:
predicted_labels = [index_to_label[pred] for pred in predictions]
print(predicted_labels)
print(len(predicted_labels))
with open(output_path, 'w') as f:
    for label in predicted_labels:
        f.write(f"{label}\n")

['false', 'half-true', 'mostly-true', 'false', 'half-true', 'true', 'mostly-true', 'half-true', 'half-true', 'mostly-true', 'mostly-true', 'half-true', 'half-true', 'mostly-true', 'barely-true', 'half-true', 'true', 'barely-true', 'half-true', 'half-true', 'barely-true', 'mostly-true', 'mostly-true', 'false', 'half-true', 'false', 'mostly-true', 'half-true', 'false', 'half-true', 'half-true', 'half-true', 'false', 'mostly-true', 'mostly-true', 'false', 'true', 'true', 'false', 'false', 'false', 'half-true', 'false', 'half-true', 'false', 'half-true', 'false', 'false', 'false', 'false', 'barely-true', 'mostly-true', 'false', 'half-true', 'half-true', 'mostly-true', 'half-true', 'false', 'false', 'mostly-true', 'half-true', 'mostly-true', 'true', 'mostly-true', 'false', 'mostly-true', 'true', 'false', 'barely-true', 'barely-true', 'true', 'half-true', 'mostly-true', 'half-true', 'mostly-true', 'true', 'false', 'false', 'half-true', 'half-true', 'false', 'half-true', 'false', 'false', 'fa

In [222]:
file= 'checker_files/bernoulli_probas_train.npy'
given= np.load(file)
print(np.log(given))
argmax_indices = np.argmax(given, axis=1)
argmax_indices= [index_to_label[z] for z in argmax_indices]
print(argmax_indices)
print(len(argmax_indices))
print(np.mean(argmax_indices==predicted_labels))
print(argmax_indices==predicted_labels)

[[ -78.64936418  -65.55233884  -70.00651476  -68.98724047  -71.93176579
   -71.72923026]
 [-108.83755008  -96.12534969  -96.07170487  -93.57188172  -97.50562238
   -98.20088982]
 [ -99.57620777  -87.38163751  -86.82345498  -87.95758968  -85.75742993
   -88.86631469]
 ...
 [-112.38087176 -104.45694306 -103.95835085  -99.03386335 -105.10612606
  -104.51048885]
 [ -72.05993847  -60.251569    -64.82133122  -64.25033752  -61.33746182
   -64.08276089]
 [-117.27653833 -117.31261369 -120.12688176 -119.74731016 -120.92763637
  -119.11899867]]
['false', 'half-true', 'mostly-true', 'false', 'half-true', 'true', 'mostly-true', 'half-true', 'half-true', 'mostly-true', 'mostly-true', 'half-true', 'half-true', 'mostly-true', 'barely-true', 'half-true', 'true', 'barely-true', 'half-true', 'half-true', 'barely-true', 'mostly-true', 'mostly-true', 'false', 'half-true', 'false', 'mostly-true', 'half-true', 'false', 'half-true', 'half-true', 'half-true', 'false', 'mostly-true', 'mostly-true', 'false', 'tr

In [223]:
test_df= pd.read_csv(test_data_path,header=None,sep='\t',quoting=3)
test_x= test_df[2]
test_y= test_df[1]

In [224]:
def create_test_feature_matrix(processed_texts, word_to_index):
    vocab_size = len(word_to_index)
    
    feature_matrix = np.zeros((len(processed_texts), vocab_size), dtype=int)
    
    unseen_word_count = np.zeros(len(processed_texts), dtype=int)
    
    for i, text in enumerate(processed_texts):
        unique_unseen_words = set() 
        for word in text:
            if word in word_to_index:
                feature_matrix[i, word_to_index[word]] = 1
            else:
                unique_unseen_words.add(word)  
        unseen_word_count[i] = len(unique_unseen_words)

    return feature_matrix, unseen_word_count


test_x= test_x.apply(preprocess_text)
X_test, unseen = create_test_feature_matrix(test_x, word_to_index)

test_predictions = model.predict_test(X_test,unseen)
test_predictions = [index_to_label[pred] for pred in test_predictions]
test_accuracy = np.mean(test_predictions == test_y)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')


[[ -52.00333246  -40.48015166  -42.2789812   -39.73408057  -39.67868227
   -41.69512625]
 [ -88.74491151  -80.66595967  -81.4007032   -80.31126924  -81.53884614
   -80.195615  ]
 [-110.90424     -99.38551043 -101.98215496 -101.44478113 -100.40857729
  -102.55482507]
 ...
 [ -77.09941068  -65.20811304  -66.96406008  -64.38311305  -65.83911869
   -65.68954241]
 [-131.80738097 -120.64328609 -121.99654747 -119.85629222 -123.12749562
  -121.91290812]
 [ -89.97738822  -75.9007393   -80.36643055  -78.00530307  -78.47179924
   -80.86988416]]
Test Accuracy: 24.22%


In [225]:
print(test_predictions)

['mostly-true', 'true', 'false', 'false', 'half-true', 'half-true', 'mostly-true', 'false', 'false', 'false', 'half-true', 'false', 'mostly-true', 'half-true', 'barely-true', 'false', 'false', 'true', 'mostly-true', 'false', 'half-true', 'false', 'false', 'half-true', 'half-true', 'false', 'false', 'false', 'mostly-true', 'mostly-true', 'half-true', 'half-true', 'true', 'half-true', 'false', 'mostly-true', 'half-true', 'false', 'half-true', 'half-true', 'half-true', 'half-true', 'half-true', 'mostly-true', 'half-true', 'half-true', 'half-true', 'barely-true', 'barely-true', 'half-true', 'barely-true', 'mostly-true', 'half-true', 'half-true', 'false', 'half-true', 'half-true', 'half-true', 'false', 'mostly-true', 'false', 'mostly-true', 'false', 'barely-true', 'barely-true', 'half-true', 'half-true', 'false', 'false', 'half-true', 'half-true', 'half-true', 'half-true', 'half-true', 'barely-true', 'false', 'half-true', 'false', 'half-true', 'mostly-true', 'false', 'false', 'false', 'half

In [226]:
print(test_y)

0       barely-true
1        pants-fire
2             false
3         half-true
4         half-true
           ...     
1279      half-true
1280    mostly-true
1281           true
1282          false
1283    barely-true
Name: 1, Length: 1284, dtype: object


In [227]:
valfile= 'checker_files/bernoulli_probas_test.npy'
givenval= np.load(valfile)
print(np.log(givenval))
argmax_indices2 = np.argmax(givenval, axis=1)
argmax_indices2= [index_to_label[z] for z in argmax_indices2]
print(argmax_indices2)
print(len(argmax_indices2))
print(len(test_predictions))
print(argmax_indices2==test_predictions)
print(np.mean(np.array(argmax_indices2)==np.array(test_predictions)))

[[ -52.00333246  -40.48015166  -42.2789812   -39.73408057  -39.67868227
   -41.69512625]
 [ -88.74491151  -80.66595967  -81.4007032   -80.31126924  -81.53884614
   -80.195615  ]
 [-110.90424     -99.38551043 -101.98215496 -101.44478113 -100.40857729
  -102.55482507]
 ...
 [ -77.09941068  -65.20811304  -66.96406008  -64.38311305  -65.83911869
   -65.68954241]
 [-131.80738097 -120.64328609 -121.99654747 -119.85629222 -123.12749562
  -121.91290812]
 [ -89.97738822  -75.9007393   -80.36643055  -78.00530307  -78.47179924
   -80.86988416]]
['mostly-true', 'true', 'false', 'false', 'half-true', 'half-true', 'mostly-true', 'false', 'false', 'false', 'half-true', 'false', 'mostly-true', 'half-true', 'barely-true', 'false', 'false', 'true', 'mostly-true', 'false', 'half-true', 'false', 'false', 'half-true', 'half-true', 'false', 'false', 'false', 'mostly-true', 'mostly-true', 'half-true', 'half-true', 'true', 'half-true', 'false', 'mostly-true', 'half-true', 'false', 'half-true', 'half-true', 'h

In [228]:
for i in range(1284):
    if argmax_indices2[i]!=test_predictions[i]:
        print(i)